# Prototyping Notebook

Prototyping for data loading, retrievals and evals



In [14]:
import sys
import os
import pprint
from data_loader import DataLoader

pp = pprint.PrettyPrinter(width=80)  # wrap after 80 characters

# Use current working directory instead of __file__
notebook_dir = os.getcwd()

# Adjust path to point to base_folder (relative to where you're running the notebook)
base_folder = os.path.abspath(os.path.join(notebook_dir, ".."))  # or adjust accordingly

# Add base_folder to sys.path
if base_folder not in sys.path:
    sys.path.append(base_folder)

# Use the loader
data_loader = DataLoader("pd_blogs")
loaded_data = data_loader.load_data()

#pp.pprint(loaded_data[1])
pp.pprint(len(loaded_data))



473


Create Vector Store

In [38]:

import vector_stores
import importlib
importlib.reload(vector_stores)
from vector_stores import VectorStoresManager

vector_stores_manager = VectorStoresManager(
    MODE="baseline",
    loaded_data=loaded_data,
    chunk_config={"enabled": True, "params": {"chunk_size": 2000, "chunk_overlap": 500}},
    embeddings_model_name="text-embedding-3-small",
    chat_model="gpt-4.1-nano",
    collection_name="Rag Loaded Data Baseline"
)

vectorstore = vector_stores_manager.get_base_vectorstore()    


In [39]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [40]:
results =  retriever.invoke("What is the cure for whinning")

pp.pprint(results)

[Document(metadata={'source': '/home/jsmejia/the_ai_eng_bootcamp/code/aim_certification_challenge/data/pd_blogs/data_toddlers_1p5-3/015_The Cure for Whining.txt', '_id': 'dd219f091c944ddfad94faafd3eee654', '_collection_name': 'Rag Loaded Data Baseline'}, page_content='The Cure for Whining'),
 Document(metadata={'source': '/home/jsmejia/the_ai_eng_bootcamp/code/aim_certification_challenge/data/pd_blogs/data_preschoolers_3-5/004_The Cure for Whining.txt', '_id': '471ab642b45b4af8bbd9613e5e54f978', '_collection_name': 'Rag Loaded Data Baseline'}, page_content='The Cure for Whining'),
 Document(metadata={'source': '/home/jsmejia/the_ai_eng_bootcamp/code/aim_certification_challenge/data/pd_blogs/data_toddlers_1p5-3/015_The Cure for Whining.txt', '_id': '60acd29e86a1478d8b38f7cdac3a8cd5', '_collection_name': 'Rag Loaded Data Baseline'}, page_content='let off stress by simply having a meltdown, but as they get older they gain more self-control, and begin to whine instead. Be kind in response 

In [37]:
print(loaded_data[0].keys())

AttributeError: 'Document' object has no attribute 'keys'